# Intro to Notebook
This notebook was made so I can learn how to have more control over GPUs. Most important to me is creating virtual devices since my machine only has one physical GPU. My GPU is an Nvidia GTX 960 4GB.

# Use a GPU
Mostly following along with Tensorflow guide: https://www.tensorflow.org/guide/gpu

In [1]:
import tensorflow as tf

In [2]:
tf.debugging.set_log_device_placement(True) # to display which devices operations and tensors are assigned to

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


### Logging Device Placement
After setting tf.debugging.set_log_device_placement(True) above, when you run code, you can see which device operations and tensors are assigned to.

In [4]:
# should see this code executed on GPU:0
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


### Manual Device Placement
If you have a good reason to do so, you can also explicitly dictate on which device you would like for the operation to run on.

In [5]:
# let's run the operation on the CPU (unless we explicitly tell it to, tf will first look for GPU)
with tf.device("/CPU:0"):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a,b) # this is run on cpu
d = tf.matmul(a, b) # this is run on gpu

print(c)
print(d)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


### Limiting GPU Memory Growth: Specific GPU(s)

In [9]:
# If I had multiple GPUs on my machine, this would limit tensorflow to only use the first one.
# NOTE: this has to be run first before GPUs are initialized (before allocating tensors or doing any operations)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
# Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


### Limiting GPU Memory Growth: Memory Growth

In [2]:
# Turn on memory growth: allocate only memory as needed, and then expand
# TODO: look into what exactly this does
# NOTE: this has to be run first before GPUs are initialized (before allocating tensors or doing any operations)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus: # memory growth must be the same in all GPUs
            tf.config.experimental.set_memory_growth(gpu, True)
            
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


### Limiting GPU Memory Growth: Virtual GPU

In [3]:
# Configure a virtual GPU with a set limit of memory
# NOTE: this has to be run first before GPUs are initialized (before allocating tensors or doing any operations)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], # get first gpu
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)] # set its memory limit
        )
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
tf.config.experimental.get_virtual_device_configuration(gpus[0]) # see the memory_limit of the set virtual device. None is returned if no virtual devices have been allocated

[LogicalDeviceConfiguration(memory_limit=1024, experimental_priority=None)]

I'm skipping the section on using a single GPU on a multi-GPU system

### Using Multiple GPUs
Here is where we look at how to simulate multiple GPUs on one device

In [3]:
# NOTE: this has to be run first before GPUs are initialized (before allocating tensors or doing any operations)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]),
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPU, 2 Logical GPUs


In [4]:
tf.config.experimental.get_virtual_device_configuration(gpus[0])

[LogicalDeviceConfiguration(memory_limit=1024, experimental_priority=None),
 LogicalDeviceConfiguration(memory_limit=1024, experimental_priority=None)]

In [5]:
# Make sure to have first run tf.debugging.set_log_device_placement(True) to see allocation

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    inputs = tf.keras.layers.Input(shape=(1,))
    predictions = tf.keras.layers.Dense(1)(inputs)
    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
    model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))
    


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/

In [6]:
# Make sure to have first run tf.debugging.set_log_device_placement(True) to see allocation

gpus = tf.config.experimental.list_logical_devices('GPU')
if gpus:
    # Replicate your computation on multiple GPUs
    c = []
    for gpu in gpus:
#         print(gpu)
        with tf.device(gpu.name):
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            z = tf.matmul(a,b)
#             c.append(tf.matmul(a, b))
            c.append(z)

    with tf.device('/CPU:0'):
        matmul_sum = tf.add_n(c) # execute on CPU

    matmul_sum_gpu = tf.add_n(c) # execute on GPU
    
    print(matmul_sum)
    print(matmul_sum_gpu)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AddN in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddN in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[ 44.  56.]
 [ 98. 128.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 44.  56.]
 [ 98. 128.]], shape=(2, 2), dtype=float32)


# Compare Run Times of NN on diff num of Logical Devices

Inspired by this Medium article: https://github.com/sayakpaul/tf.keras-Distributed-Training/blob/master/Single_GPU_Training.ipynb

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import time
from datetime import datetime

In [2]:
# tf.debugging.set_log_device_placement(True)

In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512)]
        )
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 6 Logical GPUs


In [4]:
tf.config.experimental.get_virtual_device_configuration(gpus[0])

[LogicalDeviceConfiguration(memory_limit=512, experimental_priority=None),
 LogicalDeviceConfiguration(memory_limit=512, experimental_priority=None),
 LogicalDeviceConfiguration(memory_limit=512, experimental_priority=None),
 LogicalDeviceConfiguration(memory_limit=512, experimental_priority=None),
 LogicalDeviceConfiguration(memory_limit=512, experimental_priority=None),
 LogicalDeviceConfiguration(memory_limit=512, experimental_priority=None)]

In [3]:
datasets, info = tfds.load(name="cats_vs_dogs", split=["train[:60%]","train[60%:80%]","train[80%:]"], with_info=True, as_supervised=True)
# datasets, info = tfds.load(name="cats_vs_dogs", split=["train[:120]","train[120:140]","train[140:160]"], with_info=True, as_supervised=True) # TODO: split into train/val

In [4]:
info

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    version=4.0.0,
    description='A large set of images of cats and dogs.There are 1738 corrupted images that are dropped.',
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=23262,
    splits={
        'train': 23262,
    },
    supervised_keys=('image', 'label'),
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
    title = {Asirra: A CAPTCHA that Exploits Interest-Aligned Manual Image Categorization},
    booktitle = {Proceedings of 14th ACM Conference on Computer and Communications Security (CCS)},
    ye

In [7]:
# train_data = datasets["train"]
train_data = datasets[0]
dev_data = datasets[1]
test_data = datasets[2]

In [8]:
strategy = tf.distribute.MirroredStrategy()
strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5')


6

In [9]:
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 4
# BATCH_SIZE_PER_REPLICA = 64

BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [10]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.resize(image, (224,224)) # make sure all images are the same size
    return image, label

In [11]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()


# train_name = datetime.now().strftime("%Y%m%d-%H%M%S")
# logdir = "logs/" + train_name

# TODO: try this with lambda callback
class EpochTime(tf.keras.callbacks.Callback):
    def __init__(self):
        self.start = None
        self.end = None

    def on_epoch_begin(self, epoch, logs=None):
        self.start = tf.timestamp()

    def on_epoch_end(self, epoch, logs=None):
        self.end = tf.timestamp()
        print("\nEpoch {} Time: {}".format(epoch, self.end - self.start))
        tf.summary.scalar('epoch time', data=(self.end - self.start), step=epoch)
        
        
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [12]:
train_dataset = train_data.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dev_dataset = dev_data.map(scale).cache().batch(BATCH_SIZE)
test_dataset = test_data.map(scale).batch(BATCH_SIZE)

In [13]:
def create_model(input_shape):
    X_input = tf.keras.layers.Input(input_shape)
    
    X = tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), activation="relu")(X_input)
    X = tf.keras.layers.MaxPooling2D()(X)
    X = tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), activation="relu")(X)
    X = tf.keras.layers.MaxPooling2D()(X)
    X = tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), activation="relu")(X)
    X = tf.keras.layers.MaxPooling2D()(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(64, activation="relu")(X)
    X = tf.keras.layers.Dense(1)(X)
    
    model = tf.keras.models.Model(inputs=X_input, outputs=X)
    
    return model

In [14]:
with strategy.scope():
    model = create_model((224,224,3))
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=["accuracy"])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 32)       

In [15]:
callbacks = [
    tensorboard_callback,
    EpochTime()
]

In [16]:
start = time.time()
model.fit(train_dataset, validation_data=dev_dataset, epochs=10, callbacks=callbacks)
train_time = time.time() - start
print("\nTrain Time:", train_time)

Epoch 1/10
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5').


...

  2/582 [..............................] - ETA: 5:23 - loss: 1.0260 - accuracy: 0.3542WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.3218s vs `on_train_batch_end` time: 0.6543s). Check your callbacks.


561/582 [===========================>..] - ETA: 3s - loss: 0.6065 - accuracy: 0.6386

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\eddie\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-6a52a68cf09d>", line 2, in <module>
    model.fit(train_dataset, validation_data=dev_dataset, epochs=10, callbacks=callbacks)
  File "c:\users\eddie\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "c:\users\eddie\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1098, in fit
    tmp_logs = train_function(iterator)
  File "c:\users\eddie\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\eager\def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "c:\users\eddie\appdata\local\programs\python\python3

TypeError: object of type 'NoneType' has no len()

In [19]:
# tf.keras.backend.clear_session() # doesn't do anything